# Blibliotecas necesarias

In [ ]:
!pip install openai

# Configuración API KEY

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = <completar con api key openai>

# mapbox token (access_token)
os.environ["MAPBOX_API_KEY"] = <completar con api key mapbox>

# openweather token (appid)
os.environ["OPENWEATHER_API_KEY"] = <completar con api key openweather>


# Crear un asistente

No es necesario hacerlo por código, también se puede realizar desde platform.openai.com

In [ ]:
#assistant = client.beta.assistants.create(
#    name="Math Tutor",
#    instructions="You are a personal math tutor. Write and run code to answer math questions.",
#    tools=[{"type": "code_interpreter"}],
#    model="gpt-4-1106-preview"
#)


# Usar asistente







## Indicar el ID del asistente

In [ ]:
from openai import OpenAI

client = OpenAI()
assistant_id = <completar con id de asistente>

## Función geo_code para obtener la latitud y longitud

In [ ]:
import requests
import os

def get_geocode(search_text):
  api_key = os.getenv('MAPBOX_API_KEY')
  url = f"https://api.mapbox.com/geocoding/v5/mapbox.places/{search_text}.json?access_token={api_key}"
  response = requests.get(url=url)

  data = response.json()

  nombre_lugar = data['features'][0]['place_name']
  latitud = data['features'][0]['center'][1]
  longitud = data['features'][0]['center'][0]

  return f"""Los datos obtenidos para la búsqueda del texto {search_text} fueron:\n
  Lugar: {nombre_lugar} \n
  Latitud: {latitud}    \n
  Longitud: {longitud}  \n
  """
  #return data

get_geocode("barcelona")

'Los datos obtenidos para la búsqueda del texto barcelona fueron:\n\n  Lugar: Barcelona, Barcelona, Spain \n\n  Latitud: 41.382894    \n\n  Longitud: 2.177432  \n\n  '

## Función get_weather para obtener el clima


In [ ]:
def get_weather(lat, lon, units):
  api_key = os.getenv('OPENWEATHER_API_KEY')
  url = f"https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&units={units}&appid={api_key}"
  response = requests.get(url=url)

  data = response.json()

  temperatura = data['main']['temp']
  sensasion_termica = data['main']['feels_like']
  temp_minima = data['main']['temp_min']
  temp_maxima = data['main']['temp_max']
  humedad = data['main']['humidity']

  return f"""Los datos obtenidos para latitud {lat} y longitud {lon} fueron:\n
  temperatura: {temperatura} \n
  sensacion termica: {sensasion_termica}    \n
  minima: {temp_minima}  \n
  maxima: {temp_maxima}  \n
  humedad: {humedad}  \n
  """
get_weather(-34.59, -58.41, 'metric')

'Los datos obtenidos para latitud -34.59 y longitud -58.41 fueron:\n\n  temperatura: 22.66 \n\n  sensacion termica: 23.25    \n\n  minima: 20.78  \n\n  maxima: 23.26  \n\n  humedad: 87  \n\n  '

## Obtener información del asistente

In [ ]:
mi_asistente = client.beta.assistants.retrieve(
    assistant_id=assistant_id
)
mi_asistente

Assistant(id='asst_fzBGqKnmlu677u95pEMMn7YO', created_at=1703039837, description=None, file_ids=[], instructions='Este asistente debe otorgar el clima para una determinada ciudad, para ello primero debes obtener la latitud y longitud de la ciudad empleando la funcion de geocodificacion, luego debe emplear la funcion de clima con los datos de dicha latitud y longitud. Tambien  puedo consultar directamente el clima de una latitud y longitud especifica.\n', metadata={}, model='gpt-3.5-turbo-1106', name='Clima IA', object='assistant', tools=[ToolFunction(function=FunctionDefinition(name='get_geocode', description='Determina la latitud y longitud', parameters={'type': 'object', 'properties': {'search_text': {'type': 'string', 'description': 'Indica la ciudad'}}, 'required': ['search_text']}), type='function'), ToolFunction(function=FunctionDefinition(name='get_weather', description='Determina el clima para una latitud y longitud', parameters={'type': 'object', 'properties': {'lat': {'type':

## Crear nuevo hilo

In [ ]:
mi_hilo = client.beta.threads.create()
mi_hilo

Thread(id='thread_QnXRPjgXsJumC4P8RUwvV05q', created_at=1703305328, metadata={}, object='thread')

## Crear un run con un prompt determinado

In [ ]:
mi_ejecucion = client.beta.threads.runs.create(
    thread_id=mi_hilo.id,
    assistant_id=mi_asistente.id,
    instructions="Puedes darme la latitud y longitud de barcelona"
)
mi_ejecucion

Run(id='run_psxjyLzOIyXFk03igLyNc9oc', assistant_id='asst_fzBGqKnmlu677u95pEMMn7YO', cancelled_at=None, completed_at=None, created_at=1703305339, expires_at=1703305939, failed_at=None, file_ids=[], instructions='Puedes darme la latitud y longitud de barcelona', last_error=None, metadata={}, model='gpt-3.5-turbo-1106', object='thread.run', required_action=None, started_at=None, status='queued', thread_id='thread_QnXRPjgXsJumC4P8RUwvV05q', tools=[ToolAssistantToolsFunction(function=FunctionDefinition(name='get_geocode', description='Determina la latitud y longitud', parameters={'type': 'object', 'properties': {'search_text': {'type': 'string', 'description': 'Indica la ciudad'}}, 'required': ['search_text']}), type='function'), ToolAssistantToolsFunction(function=FunctionDefinition(name='get_weather', description='Determina el clima para una latitud y longitud', parameters={'type': 'object', 'properties': {'lat': {'type': 'string', 'description': 'latitud'}, 'lon': {'type': 'string', 'de

## Obtener estado del RUN (ejecutar hasta que el status sea completed o requieres_action

In [ ]:
mi_ejecucion = client.beta.threads.runs.retrieve(
    thread_id=mi_hilo.id,
    run_id=mi_ejecucion.id
)
mi_ejecucion

Run(id='run_psxjyLzOIyXFk03igLyNc9oc', assistant_id='asst_fzBGqKnmlu677u95pEMMn7YO', cancelled_at=None, completed_at=None, created_at=1703305339, expires_at=1703305939, failed_at=None, file_ids=[], instructions='Puedes darme la latitud y longitud de barcelona', last_error=None, metadata={}, model='gpt-3.5-turbo-1106', object='thread.run', required_action=RequiredAction(submit_tool_outputs=RequiredActionSubmitToolOutputs(tool_calls=[RequiredActionFunctionToolCall(id='call_UbgHB5b5hnU9cfGpBHqMHgmV', function=Function(arguments='{"search_text":"Barcelona"}', name='get_geocode'), type='function')]), type='submit_tool_outputs'), started_at=1703305339, status='requires_action', thread_id='thread_QnXRPjgXsJumC4P8RUwvV05q', tools=[ToolAssistantToolsFunction(function=FunctionDefinition(name='get_geocode', description='Determina la latitud y longitud', parameters={'type': 'object', 'properties': {'search_text': {'type': 'string', 'description': 'Indica la ciudad'}}, 'required': ['search_text']})

# Función que invoca la función que proviene en el campo requires_action

In [ ]:
import json

def ejecutar_funcion(tool_call):

  if tool_call.function.name == 'get_geocode':
    search_text = json.loads(tool_call.function.arguments)["search_text"]

    output = get_geocode(search_text=search_text)

  if tool_call.function.name == 'get_weather':
    lat = json.loads(tool_call.function.arguments)["lat"]
    lon = json.loads(tool_call.function.arguments)["lon"]

    output = get_weather(lat=lat, lon=lon, units="metric")

  return {
      "tool_call_id": tool_call.id,
      "output": output
  }

tool_calls = mi_ejecucion.required_action.submit_tool_outputs.tool_calls
mi_tool_outputs = map(ejecutar_funcion, tool_calls)
mi_tool_outputs = list(mi_tool_outputs)
mi_tool_outputs

[{'tool_call_id': 'call_UbgHB5b5hnU9cfGpBHqMHgmV',
  'output': 'Los datos obtenidos para la búsqueda del texto Barcelona fueron:\n\n  Lugar: Barcelona, Barcelona, Spain \n\n  Latitud: 41.382894    \n\n  Longitud: 2.177432  \n\n  '}]

## Enviar el resultado de nuestra función nuevamente al asistente al thread /run_id

In [ ]:

run = client.beta.threads.runs.submit_tool_outputs(
  thread_id=mi_hilo.id,
  run_id=mi_ejecucion.id,
  tool_outputs=mi_tool_outputs
)

## Obtener el resultado final - chequear primero si el run esta completo

In [ ]:

messages = client.beta.threads.messages.list(
    thread_id=mi_hilo.id
)
messages.data[0].content[0]

MessageContentText(text=Text(annotations=[], value='La latitud de Barcelona es 41.382894 y la longitud es 2.177432.'), type='text')

## Versión unificada crea el hilo, el run y ejecuta las funciones propias en caso que se requiera

In [ ]:
import time
# caso completo obtener clima

mi_hilo = client.beta.threads.create()

mi_ejecucion = client.beta.threads.runs.create(
    thread_id=mi_hilo.id,
    assistant_id=mi_asistente.id,
    #instructions="Puedes darme el clima de la latitud 42.5 y longitud 2.17"
    instructions="Puedes darme el clima de barcelona"
)

contador = 0

while contador < 5:

  mi_ejecucion = client.beta.threads.runs.retrieve(
      thread_id=mi_hilo.id,
      run_id=mi_ejecucion.id
  )

  print(f"log: status:{mi_ejecucion.status} - accion requerida:{mi_ejecucion.required_action} - {mi_ejecucion}")
  print("---------------------")

  if mi_ejecucion.status in ["completed", "cancelled", "failed", "expired"]:
    break

  if mi_ejecucion.status == "requires_action":
    tool_calls = mi_ejecucion.required_action.submit_tool_outputs.tool_calls
    mi_tool_outputs = map(ejecutar_funcion, tool_calls)
    mi_tool_outputs = list(mi_tool_outputs)

    run = client.beta.threads.runs.submit_tool_outputs(
      thread_id=mi_hilo.id,
      run_id=mi_ejecucion.id,
      tool_outputs=mi_tool_outputs
    )
    contador = 0 #como vuelve a invocar con resultados entonces en contador va a cero

  time.sleep(5)
  contador += 1

if mi_ejecucion.status == "completed":
  messages = client.beta.threads.messages.list(
    thread_id=mi_hilo.id
  )
  print(messages.data[0].content[0])


log: status:in_progress - accion requerida:None - Run(id='run_L16ELk56Oq0dXYUASkJ8HyNq', assistant_id='asst_fzBGqKnmlu677u95pEMMn7YO', cancelled_at=None, completed_at=None, created_at=1703305725, expires_at=1703306325, failed_at=None, file_ids=[], instructions='Puedes darme el clima de barcelona', last_error=None, metadata={}, model='gpt-3.5-turbo-1106', object='thread.run', required_action=None, started_at=1703305725, status='in_progress', thread_id='thread_BfuVB3V8k6he4YB283aLhw9l', tools=[ToolAssistantToolsFunction(function=FunctionDefinition(name='get_geocode', description='Determina la latitud y longitud', parameters={'type': 'object', 'properties': {'search_text': {'type': 'string', 'description': 'Indica la ciudad'}}, 'required': ['search_text']}), type='function'), ToolAssistantToolsFunction(function=FunctionDefinition(name='get_weather', description='Determina el clima para una latitud y longitud', parameters={'type': 'object', 'properties': {'lat': {'type': 'string', 'descrip